In [1]:
import os
import numpy as np
from keras.models import Sequential # To initialise the nn as a sequence of layers
from keras.layers import Convolution2D # To make the convolution layer for 2D images
from keras.layers import MaxPooling2D # 
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.callbacks import CSVLogger
from keras.optimizers import RMSprop
from keras.layers import BatchNormalization
from keras.optimizers import Adam
from keras.models import load_model
from keras.callbacks import ModelCheckpoint

csv=CSVLogger("1_rmsprop.log")
filepath="weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')


# Initialising the CNN
classifier = Sequential()

# Step 1 - Convolution
classifier.add(Convolution2D(32,(2,2),input_shape = (224,224,1), activation = 'relu',strides=2,name='convo1'))
classifier.add(Convolution2D(64,(3,3), activation = 'relu',name='convo2'))
# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2,2)))

# Step 1 - Convolution
classifier.add(Convolution2D(64,(3,3),activation = 'relu',name='convo3'))
# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2,2)))
# Step 3 - Flattening
classifier.add(Convolution2D(64,(3,3),activation = 'relu',name='convo4'))
# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2,2)))

classifier.add(Flatten())
classifier.add(BatchNormalization(scale = False))
classifier.add(Dropout((0.5)))
classifier.add(Dense(1024, activation = 'relu'))
classifier.add(BatchNormalization(scale = False))
classifier.add(Dropout((0.4)))
classifier.add(Dense(20, activation = 'softmax'))


C:\Users\dhruv\Anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
classifier.compile(optimizer = 'rmsprop', loss = 'categorical_crossentropy', metrics = ['accuracy'])

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)

curr_path = os.getcwd()
basefolder = os.path.dirname(curr_path)
train_folder = os.path.join(basefolder, "Dataset\\Train")
test_folder = os.path.join(basefolder, "Dataset\\dev")

train_set = train_datagen.flow_from_directory(train_folder,target_size=(224, 224),batch_size=32,class_mode='categorical',color_mode='grayscale')

test_set = test_datagen.flow_from_directory(test_folder,target_size=(224, 224),batch_size=32,class_mode='categorical',color_mode='grayscale')

Found 2600 images belonging to 20 classes.
Found 200 images belonging to 20 classes.


In [3]:
history = classifier.fit_generator(train_set,steps_per_epoch=2600,epochs=10,validation_data=test_set,validation_steps=200,callbacks=[csv],verbose=2)

Epoch 1/10
 - 426s - loss: 0.3979 - acc: 0.8837 - val_loss: 4.4721 - val_acc: 0.3994
Epoch 2/10
 - 355s - loss: 0.0356 - acc: 0.9885 - val_loss: 3.7177 - val_acc: 0.4649
Epoch 3/10
 - 312s - loss: 0.0239 - acc: 0.9922 - val_loss: 5.2297 - val_acc: 0.4752
Epoch 4/10
 - 309s - loss: 0.0206 - acc: 0.9933 - val_loss: 3.8623 - val_acc: 0.4907
Epoch 5/10
 - 310s - loss: 0.0237 - acc: 0.9925 - val_loss: 4.5709 - val_acc: 0.5152
Epoch 6/10
 - 316s - loss: 0.0239 - acc: 0.9925 - val_loss: 4.1092 - val_acc: 0.4303
Epoch 7/10
 - 317s - loss: 0.0186 - acc: 0.9938 - val_loss: 3.9209 - val_acc: 0.5204
Epoch 8/10
 - 318s - loss: 0.0158 - acc: 0.9952 - val_loss: 4.7391 - val_acc: 0.4637
Epoch 9/10
 - 308s - loss: 0.0146 - acc: 0.9956 - val_loss: 4.0228 - val_acc: 0.4843
Epoch 10/10
 - 311s - loss: 0.0165 - acc: 0.9949 - val_loss: 3.8844 - val_acc: 0.5393


In [4]:
classifier.save('1_rmsprop_new.h5')